In [1]:
from pyspark.sql.functions import from_json, col, to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import SparkSession

In [2]:
def write_to_mysql(df, batch_id):
    df.write \
        .format("jdbc") \
        .option("driver","com.mysql.cj.jdbc.Driver") \
        .option("url", "jdbc:mysql://127.0.0.1:3306/spark_db") \
        .option("dbtable", "customer_search") \
        .option("user", "root") \
        .option("password", "root") \
        .save()
    df.show()

In [3]:
spark = SparkSession \
        .builder \
        .master("local[3]") \
        .appName("Stream Table Join Demo") \
        .config("spark.streaming.stopGracefullyOnShutdown", "true") \
        .config("spark.sql.shuffle.partitions", 2) \
        .config("spark.jars", "mysql-connector-java-8.0.13.jar") \
        .getOrCreate()

In [4]:
search_schema = StructType([
        StructField("id", StringType()),
        StructField("customer_id", StringType()),
        StructField("customer_name", StringType()),
        StructField("product_searched", StringType()),
        StructField("search_date", StringType()),
        StructField("country_name", StringType()),
        StructField("state", StringType())
    ])

In [5]:
kafka_source_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:9092") \
        .option("subscribe", "product-customer-qty") \
        .option("startingOffsets", "earliest") \
        .option("failOnDataLoss", False) \
        .load()

In [6]:
value_df = kafka_source_df.select(from_json(col("value").cast("string"), search_schema).alias("value"))

In [7]:
prod_customer_df = value_df.select("value.*") \
        .withColumn("search_date", to_timestamp(col("search_date"), "yyyy-MM-dd HH:mm:ss"))

In [8]:
prod_customer_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- product_searched: string (nullable = true)
 |-- search_date: timestamp (nullable = true)
 |-- country_name: string (nullable = true)
 |-- state: string (nullable = true)



In [9]:
output_df = prod_customer_df.select(col("id"), col("customer_id"), col("customer_name"), col("product_searched"), 
                             col("search_date"), col("country_name"),col("state"))
#                             .withColumn("idNum", col("customer_id").cast(IntegerType())) \
#                             .withColumn("id",    col("id").cast(IntegerType()))

In [10]:
# #to view the data in the console
# notification_writer_query = output_df.writeStream \
#         .format("console") \
#         .outputMode("append") \
#         .option("truncate", "false") \
#         .option("checkpointLocation", "./checkpoints/cassandra-proj/") \
#         .start()

# # notification_writer_query.awaitTermination()

In [11]:
#Aggregations Val

# agg_output_df = output_df.groupBy("country_name", "product_searched")\
#       .agg(count("id")).alias("search_qty")

# agg_search_locations_df = output_df.groupBy("name")\
#      .agg(sum("idNum"), count("id"))

In [12]:
query = output_df.writeStream \
                 .foreachBatch(write_to_mysql) \
                 .outputMode("update") \
                 .option("checkpointLocation", "./checkpoints/mysql-proj") \
                 .trigger(processingTime="1 minute") \
                 .start()

In [13]:
query.awaitTermination()

StreamingQueryException: Query [id = 64d46821-b68d-4e43-8632-5d8f7cbf7d97, runId = 0b3b8cd9-72e2-4c10-9572-9d5a0413f030] terminated with exception: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "D:\bigdata\spark-3.3.2-bin-hadoop3\python\lib\py4j-0.10.9.5-src.zip\py4j\clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "D:\bigdata\spark-3.3.2-bin-hadoop3\python\pyspark\sql\utils.py", line 276, in call
    raise e
  File "D:\bigdata\spark-3.3.2-bin-hadoop3\python\pyspark\sql\utils.py", line 273, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "C:\Users\EliasEvertz\AppData\Local\Temp\ipykernel_18744\4182604009.py", line 9, in write_to_mysql
    .save()
  File "D:\bigdata\spark-3.3.2-bin-hadoop3\python\pyspark\sql\readwriter.py", line 966, in save
    self._jwrite.save()
  File "D:\bigdata\spark-3.3.2-bin-hadoop3\python\lib\py4j-0.10.9.5-src.zip\